<a href="https://colab.research.google.com/github/HEM2058/sentinelhub_remote_sensing/blob/main/RVI_NDVI_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Step 1: Install required packages


# Step 2: Import libraries
import numpy as np
import json
from sentinelhub import SHConfig, SentinelHubRequest, MimeType, CRS, BBox, DataCollection,SentinelHubCatalog
import requests
from datetime import datetime, timedelta
from shapely.geometry import shape

config = SHConfig()
config.sh_client_id = '32b9acce-d3a2-4a47-b50e-ea52a441ab04'
config.sh_client_secret = 'D2FWFRrsh7MZTlbjqzS3lqlaTU1RM38j'

# Step 3: Function to find the nearest available date for RVI (Sentinel-1) and NDVI (Sentinel-2)
def find_nearest_date(bbox, input_date, data_collection):
    catalog = SentinelHubCatalog(config=config)

    # Define the search time interval ±10 days around the input date
    search_time_interval = (
        (input_date - timedelta(days=10)).strftime("%Y-%m-%d"),
        (input_date + timedelta(days=10)).strftime("%Y-%m-%d")
    )

    # Search for Sentinel-1 (RVI) data within the time interval
    search_iterator = catalog.search(
        collection=data_collection,
        bbox=bbox,
        time=search_time_interval,
        limit=1,
        sort=["date"]
    )

    search_results = list(search_iterator)

    if search_results:
        # Return the nearest available date
        return search_results[0]['properties']['datetime'].split("T")[0]  # Extract the date
    else:
        raise ValueError(f"No available data found within ±10 days for {data_collection}.")

# Step 4: Function to get Sentinel-1 RVI data
def get_rvi_data(bbox, date):
    evalscript_s1 = """
    //VERSION=3

    function setup() {
      return {
        input: ["VV", "VH", "dataMask"],
        output: { bands: 1, sampleType: "FLOAT32" }
      };
    }

    function evaluatePixel(sample) {
      let q = sample.VH / sample.VV;  // VH/VV ratio
      let N = q * (q + 3);
      let D = (q + 1) * (q + 1);
      let rvi = 1 - (N / D);
      return [rvi];
    }
    """
    request_s1 = SentinelHubRequest(
        evalscript=evalscript_s1,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL1,
                time_interval=(date, date),
            ),
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF),
        ],
        bbox=bbox,
        size=[512, 354.253],
        config=config,
    )

    response_s1 = request_s1.get_data()
    rvi_data = response_s1[0]
    return np.array(rvi_data, dtype=np.float32)

# Step 5: Function to get Sentinel-2 NDVI data
def get_ndvi_data(bbox, date):
    evalscript_s2 = """
    //VERSION=3

    function setup() {
      return {
        input: ["B04", "B08", "dataMask"],  // Red (B4) and NIR (B8) bands
        output: { bands: 1, sampleType: "FLOAT32" }
      };
    }

    function evaluatePixel(sample) {
      let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04);  // NDVI formula
      return [ndvi];
    }
    """
    request_s2 = SentinelHubRequest(
        evalscript=evalscript_s2,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(date, date),
            ),
        ],
        responses=[
            SentinelHubRequest.output_response('default', MimeType.TIFF),
        ],
        bbox=bbox,
        size=[512, 354.253],
        config=config,
    )

    response_s2 = request_s2.get_data()
    ndvi_data = response_s2[0]
    return np.array(ndvi_data, dtype=np.float32)

# Step 6: Correlation calculation between RVI and NDVI for each pixel
def calculate_pixel_correlation(rvi_data, ndvi_data):
    # Ensure both arrays have the same shape
    if rvi_data.shape != ndvi_data.shape:
        raise ValueError("RVI and NDVI arrays have different shapes.")

    correlation_data = []
    for i in range(rvi_data.shape[0]):  # Loop over rows
        for j in range(rvi_data.shape[1]):  # Loop over columns
            # Extract pixel values for both RVI and NDVI
            rvi_pixel_value = rvi_data[i, j]
            ndvi_pixel_value = ndvi_data[i, j]

            # Handle NaN or infinite values
            if np.isnan(rvi_pixel_value) or np.isnan(ndvi_pixel_value):
                correlation = np.nan  # Use np.nan for invalid values
            elif np.isinf(rvi_pixel_value) or np.isinf(ndvi_pixel_value):
                correlation = np.nan  # Use np.nan for invalid values
            else:
                # Compute the correlation coefficient for the pixel
                correlation = np.corrcoef([rvi_pixel_value], [ndvi_pixel_value])[0, 1]

            correlation_data.append({
                "pixel": (i, j),
                "correlation": correlation
            })

    return correlation_data

# Step 7: Input geometry and date
geojson_polygon = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [[
            [83.93330352722262, 28.253973768350974],
            [83.93330352722262, 28.253659569839613],
            [83.93337632222165, 28.25319788871475],
            [83.93348551472019, 28.252934986069405],
            [83.9347594272034, 28.252357880429116],
            [83.93606973718789, 28.25212703729794],
            [83.93716166217348, 28.252210397375606],
            [83.9381807921618, 28.25294139833717],
            [83.93800608416376, 28.253909646281087],
            [83.93652106618299, 28.255153607555215],
            [83.93569848269306, 28.25565375284181],
            [83.93417706721112, 28.255224141006707],
            [83.93330352722262, 28.253973768350974]
        ]]
    }
}

# Date of interest
input_date = datetime.strptime("2024-09-16", "%Y-%m-%d")

# Step 8: Convert the polygon to a BBox
polygon = shape(geojson_polygon['geometry'])
bbox = BBox(bbox=polygon.bounds, crs=CRS.WGS84)

# Step 9: Fetch the nearest available dates for Sentinel-1 and Sentinel-2
nearest_rvi_date = find_nearest_date(bbox, input_date, DataCollection.SENTINEL1)
nearest_ndvi_date = find_nearest_date(bbox, input_date, DataCollection.SENTINEL2_L2A)

print(f"Nearest available RVI (Sentinel-1) date: {nearest_rvi_date}")
print(f"Nearest available NDVI (Sentinel-2) date: {nearest_ndvi_date}")

# Step 10: Fetch RVI and NDVI data for the nearest available dates
rvi_data = get_rvi_data(bbox, nearest_rvi_date)
ndvi_data = get_ndvi_data(bbox, nearest_ndvi_date)

# Step 11: Calculate the correlation between RVI and NDVI
correlation_data = calculate_pixel_correlation(rvi_data, ndvi_data)

# Step 12: Output the results
print("RVI Data (sample):", rvi_data[:5, :5])
print("NDVI Data (sample):", ndvi_data[:5, :5])
print("Correlation Data (sample):", correlation_data[:10])

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/catalog/1.0.0/search
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/catalog/1.0.0/search
Server response: "{"code": 400, "description": "Invalid json format, problematic key 'sort'"}"

In [1]:
!pip install sentinelhub numpy requests geojson

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=7afe106288f1b678d05397623d2542215899b4d48c467b0e964d64c12f8090bb
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm
